In [1]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))

import pickle
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

import safetensors
from accelerate import notebook_launcher

import einops

from utils.data_utils import BrainDataset, get_tokenizer, pad_token_list
from utils.train_utils import TrainConfig, run_train_model, count_parameters, load_model_weights

# from transformers import GPT2Tokenizer

# from models.bert import BrainBert, BertConfig
from models.vq_brain_per_channel import SoundStream, VAEConfig

from dataclasses import dataclass
from simple_parsing.helpers import Serializable

from safetensors.torch import load_model
import albumentations as A

import evaluate
from torchsummary import summary


c:\Users\peter\alvi\brain2text\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:datasets:PyTorch version 2.3.0+cu121 available.


In [59]:
# device='cuda'
vae_config = VAEConfig(C=256, levels=(8, 8, 6, 5))
vq_vae = SoundStream(**vae_config.to_dict())

vq_weights = r'C:\Users\peter\alvi\brain2text\weights\vq_vae_v1\step_78000_loss_0.0275.safetensors'
vq_vae = load_model_weights(vq_vae, vq_weights)

self.codebook_size 1920
self.downsample 8
load compiled weights


In [2]:
""" LOAD PRETRAINED MODEL COMPONENTS """

WHISPER_MODEL_NAME = "openai/whisper-large-v3"
WHISPER_MODEL_NAME = "openai/whisper-medium"

from transformers import WhisperTokenizer, WhisperFeatureExtractor, WhisperConfig
from transformers import GenerationConfig
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# import LORA modules
# from peft import LoraConfig, AdaLoraConfig
# from peft import LoraModel, AdaLoraModel, PeftModel, get_peft_model

from audiomentations import Compose, AddGaussianNoise, TimeStretch, LowPassFilter

# load feature/label processing engines
feature_extractor = WhisperFeatureExtractor.from_pretrained(WHISPER_MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(WHISPER_MODEL_NAME, task="transcribe")
# load model
model = WhisperForConditionalGeneration.from_pretrained(WHISPER_MODEL_NAME)
# [PCH] reducing the limit of time-length
# https://github.com/sanchit-gandhi/codesnippets/blob/main/whisper-reduce-context.ipynb
# slice first 1/3 embeddings (=10 seconds input audio)
state_dict = model.state_dict()
state_dict["model.encoder.embed_positions.weight"] = state_dict["model.encoder.embed_positions.weight"][:96//2, :]
# now load these weights back into the Whisper model, this time configured for this new seq len
if ".en" not in WHISPER_MODEL_NAME:
    config = WhisperConfig.from_pretrained(WHISPER_MODEL_NAME
                                        , max_source_positions=96//2
                                        , language = "english"
                                        , task = "transcribe"
                                        , forced_decoder_ids= None
                                        )
# if ".en" not in WHISPER_MODEL_NAME:
#     model.generation_config.language = "english"
#     model.generation_config.task = "transcribe"
#     model.generation_config.forced_decoder_ids = None

model = WhisperForConditionalGeneration(config)

model.load_state_dict(state_dict)
# from dataclasses import dataclass
# from typing import Any, Dict, List, Union'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\peter\alvi\brain2text\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


<All keys matched successfully>

In [ ]:
ids = tokenizer('love')['input_ids']
tokenizer.decode(ids)

'<|startoftranscript|><|transcribe|><|notimestamps|>love<|endoftext|>'

In [3]:

metric = evaluate.load("wer")

In [ ]:
# # window_size = 512 # [PCH] - это решение волюнтаристское - мб надо поменять
# window_size = 768 
# n_electrodes = 256


# train_transform = A.Compose([
    
#     # A.CoarseDropout(fill_value=0, p=0.5),
#     # A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
#     # A.GaussNoise(var_limit=0.005, mean=0, p=0.5),

#     A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
#                   border_mode=0, value=0, always_apply=True),
#     # A.RandomCrop(height=window_size, width=n_electrodes, always_apply=True),
#     A.Crop(x_min=0, x_max=n_electrodes, y_min=0, y_max=window_size, always_apply=True),

# ])

# test_transform = A.Compose([
#     A.PadIfNeeded(min_height=window_size, min_width=n_electrodes, position='top_left', 
#                   border_mode=0, value=0, always_apply=True),
#     A.Crop(x_min=0, x_max=n_electrodes, y_min=0, y_max=window_size, always_apply=True)
# ])

# data_path = Path(r"C:\Users\peter\alvi\brain2text\competitionData")
# # tokenizer - whisper tokenizer
# # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# MAX_TOKENS = 26
# tokenize_func = lambda text: tokenizer(text)['input_ids'] 

# train_dataset = BrainDataset(data_path / 'train', tokenize_function=tokenize_func, transform=train_transform, max_tokens=MAX_TOKENS)
# test_dataset = BrainDataset(data_path / 'test', tokenize_function=tokenize_func, transform=test_transform, max_tokens=MAX_TOKENS)

Runed processing of the  C:\Users\peter\alvi\brain2text\competitionData\train
len of the dataset: 8780
max input len 906
median len 297.0
Runed processing of the  C:\Users\peter\alvi\brain2text\competitionData\test
len of the dataset: 880
max input len 919
median len 285.0


In [4]:
from torch.utils.data import Dataset
import numpy as np
from tqdm import tqdm

def check_padding(act):
    # act shape (t, c)
    # flag = (act == 0.0)
    flag = torch.sum(act != 0.0, 1) # sum of all Falses = 0 -> it was padded
    for i in range(len(flag)-1, 0, -1):
        if flag[i] != 0:
            break
    # print(i, flag[i - 1], flag[i], flag[i+1])
    # print(act[:i+1]) - True
    # print(act[i+1:]) - Padded
    end_ind = i
    return end_ind    

class VQBrainDataset(Dataset):
    def __init__(self, braindataset, vq_vae, device): 
        self.braindataset = braindataset
        vq_vae = vq_vae.to(device) 
        self.codebook_size = vq_vae.codebook_size
        new_y = []
        pad_ind = []
        new_x = []
        i = 0
        with torch.no_grad():
            for x, y, dt in tqdm(braindataset):
                # x_tens = torch.tensor(einops.rearrange(x, 't c -> 1 t c' )).to(device)
                x_tens = x.to(device)
                x_tens = vq_vae.get_indices(x_tens) / self.codebook_size
                # x_tens = einops.rearrange(x_tens, '1 t c -> t c')
                new_y.append(y)
                new_x.append(x_tens) 
                for el in x:
                    pad_ind.append(check_padding(el))
                # i+=1    
                # if i > 2:
                #     break
            

        self.inputs = torch.cat(new_x, dim = 0).detach().cpu().numpy()
        self.targets = torch.cat(new_y, 0).detach().cpu().numpy()
        self.pad_ind = np.asarray(pad_ind, dtype=np.int32)

        lens = [s.shape[0] for s in self.inputs]

        print('len of the dataset:', len(self))
        print('max input len', np.max(lens))
        print('median len', np.median(lens))



    def __len__(self) -> int:
        return len(self.inputs)

    def __getitem__(self, idx: int):
        """
        return 
            brain with shape: [time, n_channels]
            target: [n_tokens]
            date_info: 1
        """
        
        input = self.inputs[idx].astype(np.float32)
        target = self.targets[idx]
        
        # date = self.date[idx]
        # date_idx = self.date_to_index[date]
                
        # return input, target#, date_idx
        return {
            "input_features": torch.tensor(input),
            "labels": torch.tensor(target),
        }
    
    
with open('preproc_dataset.pickle', "rb") as f:
    train_dataset = pickle.load(f)   
with open('preproc_test_dataset.pickle', "rb") as f:
    test_dataset = pickle.load(f)      

In [5]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    
    # feature_extractor: Any
    # tokenizer: Any
    device : str
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # first treat the BRAIN INPUTS (already preprocessed)
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        input_features = [torch.tensor(feature["input_features"]) for feature in features]
        # batch = {"input_features": einops.rearrange(torch.stack(input_features, 0), 'b t c -> b c t').to(self.device)}
        batch = {"input_features": einops.rearrange(torch.stack(input_features, 0), 'b t c -> b c t')}
        # batch = self.feature_extractor.pad(input_features, return_tensors="pt")
        # batch = input_features
        # print(batch)

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        label_features = [torch.tensor(feature["labels"]) for feature in features]
        # pad the labels to max length
        # labels_batch = {"input_ids": torch.stack(label_features, 0).to(self.device)}
        labels_batch = {"input_ids": torch.stack(label_features, 0)}
        # labels_batch = self.tokenizer.pad(label_features, return_tensors="pt")
        # labels = label_features

        # replace padding with -100 to ignore loss correctly
        # labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        labels = labels_batch["input_ids"]
        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        # print(labels.shape)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    # tokenizer=tokenizer,
    # feature_extractor=feature_extractor,
    device = 'cuda',
    decoder_start_token_id=model.config.decoder_start_token_id,
)

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments



In [6]:
# set a new convolutional layer to accept different number of input channels
new_conv = nn.Conv1d(256, model.model.encoder.conv1.out_channels, kernel_size=(3,), stride=(1,), padding=(1,))
model.model.encoder.conv1 = new_conv

# freeze decoder self-attention. keep cross-attention
for pn, p in model.model.decoder.named_parameters():
    if 'encoder_attn' in pn:
        p.requires_grad = True
    else:
        p.requires_grad = False

summary(model)

Layer (type:depth-idx)                             Param #
├─WhisperModel: 1-1                                --
|    └─WhisperEncoder: 2-1                         --
|    |    └─Conv1d: 3-1                            787,456
|    |    └─Conv1d: 3-2                            3,146,752
|    |    └─Embedding: 3-3                         (49,152)
|    |    └─ModuleList: 3-4                        302,284,800
|    |    └─LayerNorm: 3-5                         2,048
|    └─WhisperDecoder: 2-2                         --
|    |    └─Embedding: 3-6                         (53,109,760)
|    |    └─WhisperPositionalEmbedding: 3-7        (458,752)
|    |    └─ModuleList: 3-8                        (403,070,976)
|    |    └─LayerNorm: 3-9                         (2,048)
├─Linear: 1-2                                      (53,109,760)
Total params: 816,021,504
Trainable params: 306,221,056
Non-trainable params: 509,800,448


Layer (type:depth-idx)                             Param #
├─WhisperModel: 1-1                                --
|    └─WhisperEncoder: 2-1                         --
|    |    └─Conv1d: 3-1                            787,456
|    |    └─Conv1d: 3-2                            3,146,752
|    |    └─Embedding: 3-3                         (49,152)
|    |    └─ModuleList: 3-4                        302,284,800
|    |    └─LayerNorm: 3-5                         2,048
|    └─WhisperDecoder: 2-2                         --
|    |    └─Embedding: 3-6                         (53,109,760)
|    |    └─WhisperPositionalEmbedding: 3-7        (458,752)
|    |    └─ModuleList: 3-8                        (403,070,976)
|    |    └─LayerNorm: 3-9                         (2,048)
├─Linear: 1-2                                      (53,109,760)
Total params: 816,021,504
Trainable params: 306,221,056
Non-trainable params: 509,800,448

In [7]:
import wandb


In [7]:
# wandb.init()
experiment_path = Path('logs/whisper/petr')
experiment_path.mkdir(parents=True, exist_ok=True)

batch_size = 256
epoch_length = int(len(train_dataset) / batch_size)

training_args = Seq2SeqTrainingArguments(
    output_dir=experiment_path,  # change to a repo name of your choice
    per_device_train_batch_size=batch_size,
    # gradient_accumulation_steps=int(16 / batch_size),  # increase by 2x for every 2x decrease in batch size
    learning_rate=2.5e-5,
    num_train_epochs = 10000,
    warmup_steps=epoch_length,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    generation_max_length=32,
    save_steps=int(epoch_length / 2),
    eval_steps=int(epoch_length / 2),
    logging_steps=25,
    report_to=["wandb"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    use_cpu= False
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model.to('cuda'),
    # model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
    # pin_memory = False,

)
trainer.train()

c:\Users\peter\alvi\brain2text\.venv\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: peter_chizhov. Use `wandb login --relogin` to force relogin


  0%|          | 0/350000 [00:00<?, ?it/s]C:\Users\peter\AppData\Local\Temp\ipykernel_20708\3276194435.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_features = [torch.tensor(feature["input_features"]) for feature in features]
C:\Users\peter\AppData\Local\Temp\ipykernel_20708\3276194435.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_features = [torch.tensor(feature["labels"]) for feature in features]
c:\Users\peter\alvi\brain2text\.venv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=

{'eval_loss': 4.49925422668457, 'eval_wer': 116.31837179720152, 'eval_runtime': 39.8602, 'eval_samples_per_second': 22.077, 'eval_steps_per_second': 0.1, 'epoch': 0.49}


C:\Users\peter\AppData\Local\Temp\ipykernel_20708\3276194435.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_features = [torch.tensor(feature["input_features"]) for feature in features]
C:\Users\peter\AppData\Local\Temp\ipykernel_20708\3276194435.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_features = [torch.tensor(feature["labels"]) for feature in features]
c:\Users\peter\alvi\brain2text\.venv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to p

{'loss': 7.4011, 'grad_norm': 12.772808074951172, 'learning_rate': 1.8382352941176472e-05, 'epoch': 0.71}


...

C:\Users\peter\AppData\Local\Temp\ipykernel_20708\3276194435.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_features = [torch.tensor(feature["input_features"]) for feature in features]
C:\Users\peter\AppData\Local\Temp\ipykernel_20708\3276194435.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_features = [torch.tensor(feature["labels"]) for feature in features]
c:\Users\peter\alvi\brain2text\.venv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to p

{'loss': 0.4601, 'grad_norm': 2.1047542095184326, 'learning_rate': 2.488456021442083e-05, 'epoch': 47.14}


                                                            
  0%|          | 1666/350000 [2:29:42<261:32:46,  2.70s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_supp

{'eval_loss': 4.711363792419434, 'eval_wer': 108.9950935853171, 'eval_runtime': 39.9736, 'eval_samples_per_second': 22.015, 'eval_steps_per_second': 0.1, 'epoch': 47.6}


RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 2575904448 vs 2575904340



C:\Users\peter\AppData\Local\Temp\ipykernel_9244\3276194435.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_features = [torch.tensor(feature["input_features"]) for feature in features]
C:\Users\peter\AppData\Local\Temp\ipykernel_9244\3276194435.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_features = [torch.tensor(feature["labels"]) for feature in features]
c:\Users\peter\alvi\brain2text\.venv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to p

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=False)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)


In [ ]:
# train_dataset_new = VQBrainDataset(train_dataloader, vq_vae=vq_vae, device='cuda')
# with open('preproc_dataset.pickle', 'wb') as f:
#     pickle.dump(train_dataset_new, f) 
# # train_dataset_new

  0%|          | 0/1098 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# import pickle
# test_dataset_new = VQBrainDataset(test_dataloader, vq_vae=vq_vae, device='cuda')
# with open('preproc_test_dataset.pickle', 'wb') as f:
#     pickle.dump(test_dataset_new, f) 

100%|██████████| 110/110 [01:20<00:00,  1.37it/s]


len of the dataset: 880
max input len 96
median len 96.0


Layer (type:depth-idx)                             Param #
├─WhisperModel: 1-1                                --
|    └─WhisperEncoder: 2-1                         --
|    |    └─Conv1d: 3-1                            787,456
|    |    └─Conv1d: 3-2                            3,146,752
|    |    └─Embedding: 3-3                         (49,152)
|    |    └─ModuleList: 3-4                        302,284,800
|    |    └─LayerNorm: 3-5                         2,048
|    └─WhisperDecoder: 2-2                         --
|    |    └─Embedding: 3-6                         (53,109,760)
|    |    └─WhisperPositionalEmbedding: 3-7        (458,752)
|    |    └─ModuleList: 3-8                        (403,070,976)
|    |    └─LayerNorm: 3-9                         (2,048)
├─Linear: 1-2                                      (53,109,760)
Total params: 816,021,504
Trainable params: 306,221,056
Non-trainable params: 509,800,448


Layer (type:depth-idx)                             Param #
├─WhisperModel: 1-1                                --
|    └─WhisperEncoder: 2-1                         --
|    |    └─Conv1d: 3-1                            787,456
|    |    └─Conv1d: 3-2                            3,146,752
|    |    └─Embedding: 3-3                         (49,152)
|    |    └─ModuleList: 3-4                        302,284,800
|    |    └─LayerNorm: 3-5                         2,048
|    └─WhisperDecoder: 2-2                         --
|    |    └─Embedding: 3-6                         (53,109,760)
|    |    └─WhisperPositionalEmbedding: 3-7        (458,752)
|    |    └─ModuleList: 3-8                        (403,070,976)
|    |    └─LayerNorm: 3-9                         (2,048)
├─Linear: 1-2                                      (53,109,760)
Total params: 816,021,504
Trainable params: 306,221,056
Non-trainable params: 509,800,448

In [ ]:
model(einops.rearrange(torch.tensor(train_dataset[0][0]), 't c -> 1 c t')
      , decoder_input_ids = torch.tensor(train_dataset[0][1]))

IndexError: index out of range in self

In [ ]:
model = model.to('cuda')

In [ ]:
einops.rearrange(torch.tensor(train_dataset[0][0]).to('cuda'), 't c -> 1 c t').shape

torch.Size([1, 256, 96])

In [ ]:
model.generate(einops.rearrange(torch.tensor(train_dataset[0][0]).to('cuda'), 't c -> 1 c t'), max_new_tokens = 26)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model(einops.rearrange(torch.tensor(train_dataset[0][0]).to('cuda'), 't c -> 1 c t')
      , decoder_input_ids = torch.tensor(train_dataset[0][1]).to('cuda'))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
project_name = 'vqvae_whisper'
save_folder = Path("logs/chizhov")

train_config = TrainConfig(exp_name='vqvae_pretrain_large_whisper',
                           mixed_precision=True,
                           batch_size=2, 
                           num_workers=3,
                           pin_memory=True, 
                           eval_interval=500, 
                           learning_rate=3e-4,
                           weight_decay=0, 
                           grad_clip=5,
                           lr_decay_iters=30_000, 
                           warmup_iters=1000, 
                           project_name=project_name, 
                           save_folder=save_folder
                          )

# model = torch.compile(model)
args = (model, (train_dataset, test_dataset), train_config)
notebook_launcher(run_train_model, args, num_processes=1)

In [ ]:
model.model.encoder.conv1

Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))

In [ ]:
#  На вход в whisper encoder идет model.model.encoder.conv1.in_channels каналов
train_dataset_new[0][0].shape, train_dataset_new[0][1].shape 


((96, 256), (26,))

In [ ]:
summary(model)

Layer (type:depth-idx)                             Param #
├─WhisperModel: 1-1                                --
|    └─WhisperEncoder: 2-1                         --
|    |    └─Conv1d: 3-1                            492,800
|    |    └─Conv1d: 3-2                            4,916,480
|    |    └─Embedding: 3-3                         (1,920,000)
|    |    └─ModuleList: 3-4                        629,637,120
|    |    └─LayerNorm: 3-5                         2,560
|    └─WhisperDecoder: 2-2                         --
|    |    └─Embedding: 3-6                         66,388,480
|    |    └─WhisperPositionalEmbedding: 3-7        573,440
|    |    └─ModuleList: 3-8                        839,557,120
|    |    └─LayerNorm: 3-9                         2,560
├─Linear: 1-2                                      66,388,480
Total params: 1,609,879,040
Trainable params: 1,607,959,040
Non-trainable params: 1,920,000


Layer (type:depth-idx)                             Param #
├─WhisperModel: 1-1                                --
|    └─WhisperEncoder: 2-1                         --
|    |    └─Conv1d: 3-1                            492,800
|    |    └─Conv1d: 3-2                            4,916,480
|    |    └─Embedding: 3-3                         (1,920,000)
|    |    └─ModuleList: 3-4                        629,637,120
|    |    └─LayerNorm: 3-5                         2,560
|    └─WhisperDecoder: 2-2                         --
|    |    └─Embedding: 3-6                         66,388,480
|    |    └─WhisperPositionalEmbedding: 3-7        573,440
|    |    └─ModuleList: 3-8                        839,557,120
|    |    └─LayerNorm: 3-9                         2,560
├─Linear: 1-2                                      66,388,480
Total params: 1,609,879,040
Trainable params: 1,607,959,040
Non-trainable params: 1,920,000